In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from gensim.models import Word2Vec
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm


In [ ]:

df = pd.read_csv('fullDataset.csv')

# drop MT
# df = df[df['source'] != 'MACHINE_TRANSLATION']

df = df[['sentence', 'label']]

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zăîâșțA-ZĂÎÂȘȚ ]+', '', text)  
    return text

df['clean_sentence'] = df['sentence'].astype(str).apply(clean_text)

df['tokens'] = df['clean_sentence'].apply(lambda x: x.split())

sentences = df['tokens'].tolist()

w2v_model = Word2Vec(
    sentences,
    vector_size=100,  
    window=5,
    min_count=1,    
    workers=4,
    sg=1            
)

def get_sentence_vector(tokens, model):
    vec = []
    for word in tokens:
        if word in model.wv:
            vec.append(model.wv[word])
    if len(vec) == 0:
        return np.zeros(model.vector_size)
    else:
        return np.mean(vec, axis=0)


In [ ]:

X = np.array([get_sentence_vector(tokens, w2v_model) for tokens in df['tokens']])
y = df['label'].values


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:
svm = SVC(kernel='linear', C=1, random_state=42)
svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True) )


In [ ]:
model_name = 'dumitrescustefan/bert-base-romanian-cased-v1'  # Romanian BERT
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

def get_bert_embedding(text):
    tokens = tokenizer(text, padding='max_length', truncation=True, max_length=64, return_tensors='pt')
    tokens = {key: val.to(device) for key, val in tokens.items()}

    with torch.no_grad():
        outputs = model(**tokens)
        hidden_states = outputs.last_hidden_state  
        embedding = hidden_states.mean(dim=1).squeeze().cpu().numpy()
    return embedding

embeddings = []

print("Generare BERT embeddings...")
for sentence in tqdm(df['sentence'].tolist()):
    emb = get_bert_embedding(sentence)
    embeddings.append(emb)


In [ ]:

X = np.array(embeddings)
y = df['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

svm = SVC(kernel='linear', C=1, random_state=42)
svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True))
